<a href="https://colab.research.google.com/github/luizvalle/Chessformer/blob/main/chessformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# @title Install dependencies {display-mode: "form"}
%%shell
pip install chess
pip install zstd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:00


In [82]:
# @title Imports {display-mode: "form"}
import chess.pgn
import zstandard as zstd
import requests
import io

from pprint import pprint

## Download and prepare the data

In [83]:
DOWNLOAD_LIST = "https://database.lichess.org/standard/list.txt"
download_links = requests.get(DOWNLOAD_LIST).text.split('\n')
pprint(download_links[0:5])

['https://database.lichess.org/standard/lichess_db_standard_rated_2023-09.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-08.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-07.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-06.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-05.pgn.zst']


In [85]:
class CompressedPgnIterator:
  def __init__(self, download_link):
      dctx = zstd.ZstdDecompressor()
      response = requests.get(url=download_link, stream=True)
      reader = dctx.stream_reader(response.raw)
      self.text_stream = io.TextIOWrapper(reader, encoding='utf-8')

  def __iter__(self):
      return self

  def __next__(self):
    game = chess.pgn.read_game(self.text_stream)
    if game:
      return game
    else:
      raise StopIteration

In [ ]:
events = set()
results = set()
elos = set()
time_controls = set()
terminations = set()
for download_link in download_links:
  games = CompressedPgnIterator(download_link)
  for game in games:
    events.add(game.headers["Event"])
    results.add(game.headers["Result"])
    elos.add(game.headers["WhiteElo"])
    elos.add(game.headers["BlackElo"])
    time_controls.add(game.headers["TimeControl"])
    terminations.add(game.headers["Termination"])

In [ ]:
print(events)

In [ ]:
print(results)

In [ ]:
print(elos)

In [ ]:
print(time_controls)